In [1]:
import collections
import re
from d2l import torch as d2l

### Load the Dataset

In [2]:
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():  
    '''Load the time machine dataset into a list of text lines'''
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]
                  # [^A-Za-z] means if this character position is not an English letter
                  # change it to space

lines = read_time_machine()
print(f'# text lines: {len(lines)}')
print(lines[0])
print(lines[10])

# text lines: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


### Tokenization

In [3]:
def tokenize(lines, token='word'):
    '''Split text lines into word or character tokens(词源)'''
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('ERROR: unknown token type: ' +  token)

tokens = tokenize(lines)
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


### Build a Vocabulary: Maps a token of type string to a numeric index starting at 0

In [9]:
def count_corpus(tokens):
    '''Count the num of occurrences of each token'''
    if len(tokens) == 0 or isinstance(tokens[0], list):
        tokens = [token for line in tokens for token in line]
    # collections.Counter is to create a dict to count different tokens
    return collections.Counter(tokens)

In [10]:
class Vocab:
    '''Vocabulary for text'''
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
    # 'min_freq' = if some token num is less than 'min_freq', then drop this token
    # 'reserved_tokens' = the token that at the beginning and the end of sentence
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        counter = count_corpus(tokens)
        
        # sort the tokens, which occur most often are placed first
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1], reverse=True)
        
        # 'uniq_tokens' = save the speical tokens like unknown tokens; reserved_tokens; 
        # tokens that occur more than min_freq
        self.unk, uniq_tokens = 0, ['<unk>'] + reserved_tokens
        uniq_tokens += [
            token for token, freq in self.token_freqs
            if freqs >= min_freq and token not in uniq_tokens] 
        
        # give index then return token --> save in a list
        # give token then return index --> save in a dict
        self.idx_to_token, self.token_to_idx = [], dict()
        for token in uniq_tokens:
            self.idx_to_token.append(token)
            self.token_to_idx[token] = len(self.idx_to_token) - 1
        
        
    def __len__(self):
        # length is the num of uniq_tokens
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        # getitem is to get the index from token
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        # to_token is to get the token from index
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

    @property
    def unk(self):  # Index for the unknown token
        return 0

    @property
    def token_freqs(self):  # Index for the unknown token
        return self._token_freqs

In [6]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


In [7]:
# turn the text lines of text into a number index list
for i in [0, 10]:
    print('words:', tokens[i])
    print('indices:', vocab[tokens[i]])

words: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
indices: [1, 19, 50, 40, 2183, 2184, 400]
words: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
indices: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


### Put All together

In [12]:
def load_corpus_time_machine(max_tokens=-1):
    '''Return the token index list and vocabulary for the Time machine Dataset'''
    lines = read_time_machine()
    #tokens = tokenize(lines, 'word')
    tokens = tokenize(lines, 'char')
    vocab = Vocab(tokens)
    # now corpus is the long number
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)
# vocab = 28 because 26 letters plus space and unk

(170580, 28)